NOTES TO SELF:

Changing 'mins' into int form?
Changing dates into datetime form?

In [514]:
# Dependencies
import pandas as pd
from pathlib import Path

# CSV paths
games_details_path = Path('../data/cleaned_data/games_details_cleaned.csv')
games_path = Path('../data/cleaned_data/games_cleaned.csv')
teams_path = Path('../data/cleaned_data/teams_cleaned.csv')
players_path = Path('../data/cleaned_data/players_cleaned.csv')
ranking_path = Path('../data/cleaned_data/ranking_cleaned.csv')

# Creating dataframes
games_details = pd.read_csv(games_details_path)
games = pd.read_csv(games_path)
teams = pd.read_csv(teams_path)
players = pd.read_csv(players_path)
ranking = pd.read_csv(ranking_path)

Dealig with primary keys! Creating necessary composite keys, and making sure there are no duplicates.

PRIMARY KEYS:
games_details = GAME_ID + PLAYER ID
games = GAME_ID
teams = TEAM_ID
players = TEAM_ID + PLAYER_ID + SEASON
ranking = TEAM_ID + STANDINGSDATE

1. games_details = GAME_ID + PLAYER ID

In [515]:
# View dataframe
games_details.head(3)

,games_details_PK,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,MIN,FGM,...,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,SEASON
0,222004771629641,22200477,1610612759,SAS,San Antonio,1629641,Romeo Langford,F,18.06,1.0,...,1.0,2.0,0.0,1.0,0.0,2.0,5.0,2.0,-2.0,2022
1,222004771631110,22200477,1610612759,SAS,San Antonio,1631110,Jeremy Sochan,F,31.01,7.0,...,3.0,9.0,6.0,1.0,0.0,2.0,1.0,23.0,-14.0,2022
2,222004771627751,22200477,1610612759,SAS,San Antonio,1627751,Jakob Poeltl,C,21.42,6.0,...,3.0,4.0,1.0,1.0,0.0,2.0,4.0,13.0,-4.0,2022


In [516]:
# Making composite key (changing columns to strings or else it adds them, therefore not unique)
games_details['games_details_PK'] = games_details['GAME_ID'].astype(str
                                    ) + games_details['PLAYER_ID'].astype(str)

In [517]:
# Make sure it worked
games_details.head(1)

,games_details_PK,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,MIN,FGM,...,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,SEASON
0,222004771629641,22200477,1610612759,SAS,San Antonio,1629641,Romeo Langford,F,18.06,1.0,...,1.0,2.0,0.0,1.0,0.0,2.0,5.0,2.0,-2.0,2022


In [518]:
# Get column names so you can easily rearrange them in the next step
games_details.columns

Index(['games_details_PK', 'GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION',
       'TEAM_CITY', 'PLAYER_ID', 'PLAYER_NAME', 'START_POSITION', 'MIN', 'FGM',
       'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
       'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS',
       'PLUS_MINUS', 'SEASON'],
      dtype='object')

In [519]:
# Rearrange columns so the primary key is the first column
games_details = games_details[['games_details_PK','GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID',
       'PLAYER_NAME', 'NICKNAME', 'START_POSITION', 'COMMENT', 'MIN', 'FGM',
       'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
       'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS',
       'PLUS_MINUS']]
games_details.head(1)

KeyError: "['NICKNAME', 'COMMENT'] not in index"

In [ ]:
# Check to see if there duplicates in the primary key
games_details['games_details_PK'].value_counts()

games_details_PK
222004771629641    1
218000291626185    1
21800026203124     1
21800026203926     1
218000261628410    1
                  ..
20500732708        1
205007322058       1
205007321729       1
205007321853       1
11200005201951     1
Name: count, Length: 668339, dtype: int64

In [ ]:
# Double check that duplicates are really duplicates
games_details[games_details['games_details_PK']=='22000013201577']

,games_details_PK,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
67724,22000013201577,22000013,1610612764,WAS,Washington,201577,Robin Lopez,NaN,NaN,NaN,...,1.0,2.0,3.0,0.0,0.0,1.0,2.0,4.0,3.0,-16.0


In [ ]:
# Drop duplicates
games_details = games_details.drop_duplicates(subset=['games_details_PK'])

In [ ]:
# Make sure duplicates got dropped
games_details['games_details_PK'].value_counts()

games_details_PK
222004771629641    1
218000291626185    1
21800026203124     1
21800026203926     1
218000261628410    1
                  ..
20500732708        1
205007322058       1
205007321729       1
205007321853       1
11200005201951     1
Name: count, Length: 668339, dtype: int64

2. games = GAME_ID

In [ ]:
# View dataframe
games.head(3)

,GAME_ID_pk,GAME_DATE_EST,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,22200477,2022-12-22,Final,1610612740,1610612759,2022,1610612740,126.0,0.484,0.926,...,25.0,46.0,1610612759,117.0,0.478,0.815,0.321,23.0,44.0,1
1,22200478,2022-12-22,Final,1610612762,1610612764,2022,1610612762,120.0,0.488,0.952,...,16.0,40.0,1610612764,112.0,0.561,0.765,0.333,20.0,37.0,1
2,22200466,2022-12-21,Final,1610612739,1610612749,2022,1610612739,114.0,0.482,0.786,...,22.0,37.0,1610612749,106.0,0.470,0.682,0.433,20.0,46.0,1


In [ ]:
# Get column names
games.columns

Index(['GAME_ID_pk', 'GAME_DATE_EST', 'GAME_STATUS_TEXT', 'HOME_TEAM_ID',
       'VISITOR_TEAM_ID', 'SEASON', 'TEAM_ID_home', 'PTS_home', 'FG_PCT_home',
       'FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home', 'TEAM_ID_away',
       'PTS_away', 'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away',
       'REB_away', 'HOME_TEAM_WINS'],
      dtype='object')

In [ ]:
# Rearrange columns
games = games[['GAME_ID_pk','GAME_DATE_EST', 'GAME_STATUS_TEXT', 'HOME_TEAM_ID',
       'VISITOR_TEAM_ID', 'SEASON', 'TEAM_ID_home', 'PTS_home', 'FG_PCT_home',
       'FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home', 'TEAM_ID_away',
       'PTS_away', 'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away',
       'REB_away', 'HOME_TEAM_WINS']]
games.head(1)

,GAME_ID_pk,GAME_DATE_EST,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,22200477,2022-12-22,Final,1610612740,1610612759,2022,1610612740,126.0,0.484,0.926,...,25.0,46.0,1610612759,117.0,0.478,0.815,0.321,23.0,44.0,1


In [ ]:
# Rename PK column
games = games.rename(columns={'GAME_ID':'GAME_ID_pk'})
games.head(1)

,GAME_ID_pk,GAME_DATE_EST,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,22200477,2022-12-22,Final,1610612740,1610612759,2022,1610612740,126.0,0.484,0.926,...,25.0,46.0,1610612759,117.0,0.478,0.815,0.321,23.0,44.0,1


In [ ]:
# Check for duplicates
games['GAME_ID_pk'].value_counts()

GAME_ID_pk
22200477    1
20400082    1
20400073    1
20400079    1
20400077    1
           ..
21000429    1
21000428    1
21000427    1
21000426    1
11400001    1
Name: count, Length: 26622, dtype: int64

In [ ]:
# Double check that duplicates are really duplicates
games[games['GAME_ID_pk']==22000032]

,GAME_ID_pk,GAME_DATE_EST,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
2515,22000032,2020-12-27,Final,1610612766,1610612751,2020,1610612766,106.0,0.448,0.588,...,35.0,52.0,1610612751,104.0,0.427,0.68,0.362,23.0,46.0,1


In [ ]:
# Drop duplicates
games = games.drop_duplicates(subset=['GAME_ID_pk'])

# Make sure duplicates got dropped
games['GAME_ID_pk'].value_counts()

GAME_ID_pk
22200477    1
20400082    1
20400073    1
20400079    1
20400077    1
           ..
21000429    1
21000428    1
21000427    1
21000426    1
11400001    1
Name: count, Length: 26622, dtype: int64

In [ ]:
# Exporting cleaned data
games.to_csv('../data/cleaned_data/games_cleaned.csv',index=False,header=True)

3. teams = TEAM_ID

In [ ]:
# View dataframe
teams.head(3)

,TEAM_ID_pk,LEAGUE_ID,MIN_YEAR,MAX_YEAR,ABBREVIATION,NICKNAME,YEARFOUNDED,CITY,ARENA,ARENACAPACITY,OWNER,GENERALMANAGER,HEADCOACH,DLEAGUEAFFILIATION
0,1610612737,0,1949,2019,ATL,Hawks,1949,Atlanta,State Farm Arena,18729.0,Tony Ressler,Travis Schlenk,Lloyd Pierce,Erie Bayhawks
1,1610612738,0,1946,2019,BOS,Celtics,1946,Boston,TD Garden,18624.0,Wyc Grousbeck,Danny Ainge,Brad Stevens,Maine Red Claws
2,1610612740,0,2002,2019,NOP,Pelicans,2002,New Orleans,Smoothie King Center,NaN,Tom Benson,Trajan Langdon,Alvin Gentry,No Affiliate


In [ ]:
# Get column names
teams.columns

Index(['TEAM_ID_pk', 'LEAGUE_ID', 'MIN_YEAR', 'MAX_YEAR', 'ABBREVIATION',
       'NICKNAME', 'YEARFOUNDED', 'CITY', 'ARENA', 'ARENACAPACITY', 'OWNER',
       'GENERALMANAGER', 'HEADCOACH', 'DLEAGUEAFFILIATION'],
      dtype='object')

In [ ]:
# Rearrange columns
teams = teams[['TEAM_ID_pk','LEAGUE_ID', 'MIN_YEAR', 'MAX_YEAR', 'ABBREVIATION',
       'NICKNAME', 'YEARFOUNDED', 'CITY', 'ARENA', 'ARENACAPACITY', 'OWNER',
       'GENERALMANAGER', 'HEADCOACH', 'DLEAGUEAFFILIATION']]
teams.head(1)

,TEAM_ID_pk,LEAGUE_ID,MIN_YEAR,MAX_YEAR,ABBREVIATION,NICKNAME,YEARFOUNDED,CITY,ARENA,ARENACAPACITY,OWNER,GENERALMANAGER,HEADCOACH,DLEAGUEAFFILIATION
0,1610612737,0,1949,2019,ATL,Hawks,1949,Atlanta,State Farm Arena,18729.0,Tony Ressler,Travis Schlenk,Lloyd Pierce,Erie Bayhawks


In [ ]:
# Rename PK column
teams = teams.rename(columns={'TEAM_ID':'TEAM_ID_pk'})
teams.head(1)

,TEAM_ID_pk,LEAGUE_ID,MIN_YEAR,MAX_YEAR,ABBREVIATION,NICKNAME,YEARFOUNDED,CITY,ARENA,ARENACAPACITY,OWNER,GENERALMANAGER,HEADCOACH,DLEAGUEAFFILIATION
0,1610612737,0,1949,2019,ATL,Hawks,1949,Atlanta,State Farm Arena,18729.0,Tony Ressler,Travis Schlenk,Lloyd Pierce,Erie Bayhawks


In [ ]:
# Check for duplicates
teams['TEAM_ID_pk'].value_counts()

TEAM_ID_pk
1610612737    1
1610612738    1
1610612739    1
1610612766    1
1610612765    1
1610612764    1
1610612763    1
1610612762    1
1610612761    1
1610612760    1
1610612759    1
1610612758    1
1610612757    1
1610612756    1
1610612755    1
1610612754    1
1610612753    1
1610612752    1
1610612751    1
1610612750    1
1610612749    1
1610612748    1
1610612747    1
1610612746    1
1610612745    1
1610612743    1
1610612742    1
1610612741    1
1610612740    1
1610612744    1
Name: count, dtype: int64

In [ ]:
# Exporting cleaned data
teams.to_csv('../data/cleaned_data/teams_cleaned.csv',index=False,header=True)

4. players = TEAM_ID + PLAYER_ID + SEASON

In [ ]:
# View dataframe
players.head(3)

,players_PK,PLAYER_NAME,TEAM_ID,PLAYER_ID,SEASON
0,161061276216262202019,Royce O'Neale,1610612762,1626220,2019
1,16106127622027112019,Bojan Bogdanovic,1610612762,202711,2019
2,16106127622034972019,Rudy Gobert,1610612762,203497,2019


In [ ]:
# Making composite key 
players['players_PK'] = players['TEAM_ID'].astype(str) + players[
    'PLAYER_ID'].astype(str) + players['SEASON'].astype(str) 

In [ ]:
# Make sure it worked
players.head(1)

,players_PK,PLAYER_NAME,TEAM_ID,PLAYER_ID,SEASON
0,161061276216262202019,Royce O'Neale,1610612762,1626220,2019


In [ ]:
# Get column names
players.columns

Index(['players_PK', 'PLAYER_NAME', 'TEAM_ID', 'PLAYER_ID', 'SEASON'], dtype='object')

In [ ]:
# Rearrange columns
players = players[['players_PK', 'PLAYER_NAME', 'TEAM_ID', 'PLAYER_ID', 'SEASON']]
players.head(1)

,players_PK,PLAYER_NAME,TEAM_ID,PLAYER_ID,SEASON
0,161061276216262202019,Royce O'Neale,1610612762,1626220,2019


In [ ]:
# Check for duplicates
players['players_PK'].value_counts()

players_PK
161061276216262202019    1
16106127522007792012     1
16106127542031112012     1
16106127541011232012     1
16106127541011332012     1
                        ..
161061275816278342016    1
16106127582040382016     1
161061275816277412016    1
161061275816261612016    1
16106127625000322009     1
Name: count, Length: 7228, dtype: int64

In [ ]:
# Exporting cleaned data
players.to_csv('../data/cleaned_data/players_cleaned.csv',index=False,header=True)

5. ranking = TEAM_ID + STANDINGSDATE

In [ ]:
# View dataframe
ranking.head(3)

,ranking_PK,TEAM_ID,LEAGUE_ID,SEASON_ID,STANDINGSDATE,CONFERENCE,TEAM,G,W,L,W_PCT,HOME_RECORD,ROAD_RECORD,RETURNTOPLAY
0,16106127432022-12-22,1610612743,0,22022,2022-12-22,West,Denver,30,19,11,0.633,10-3,9-8,NaN
1,16106127632022-12-22,1610612763,0,22022,2022-12-22,West,Memphis,30,19,11,0.633,13-2,6-9,NaN
2,16106127402022-12-22,1610612740,0,22022,2022-12-22,West,New Orleans,31,19,12,0.613,13-4,6-8,NaN


In [ ]:
# Making composite key
ranking['ranking_PK'] = ranking['TEAM_ID'].astype(str) + ranking['STANDINGSDATE'] 

In [ ]:
# Make sure it worked
ranking.head(1)

,ranking_PK,TEAM_ID,LEAGUE_ID,SEASON_ID,STANDINGSDATE,CONFERENCE,TEAM,G,W,L,W_PCT,HOME_RECORD,ROAD_RECORD,RETURNTOPLAY
0,16106127432022-12-22,1610612743,0,22022,2022-12-22,West,Denver,30,19,11,0.633,10-3,9-8,NaN


In [ ]:
# Column names
ranking.columns

Index(['ranking_PK', 'TEAM_ID', 'LEAGUE_ID', 'SEASON_ID', 'STANDINGSDATE',
       'CONFERENCE', 'TEAM', 'G', 'W', 'L', 'W_PCT', 'HOME_RECORD',
       'ROAD_RECORD', 'RETURNTOPLAY'],
      dtype='object')

In [ ]:
# Rearrange columns
ranking = ranking[['ranking_PK','TEAM_ID', 'LEAGUE_ID', 'SEASON_ID', 'STANDINGSDATE', 'CONFERENCE',
       'TEAM', 'G', 'W', 'L', 'W_PCT', 'HOME_RECORD', 'ROAD_RECORD',
       'RETURNTOPLAY']]
ranking.head(1)

,ranking_PK,TEAM_ID,LEAGUE_ID,SEASON_ID,STANDINGSDATE,CONFERENCE,TEAM,G,W,L,W_PCT,HOME_RECORD,ROAD_RECORD,RETURNTOPLAY
0,16106127432022-12-22,1610612743,0,22022,2022-12-22,West,Denver,30,19,11,0.633,10-3,9-8,NaN


In [ ]:
# Check for duplicates
ranking['ranking_PK'].value_counts()

ranking_PK
16106127432022-12-22    1
16106127652011-04-16    1
16106127412011-04-16    1
16106127482011-04-16    1
16106127382011-04-16    1
                       ..
16106127632007-12-25    1
16106127602007-12-25    1
16106127502007-12-25    1
16106127592007-12-24    1
16106127492014-09-01    1
Name: count, Length: 210313, dtype: int64

In [ ]:
# Double check that duplicates are really duplicates
ranking[ranking['ranking_PK']=='16106127442020-12-28']

,ranking_PK,TEAM_ID,LEAGUE_ID,SEASON_ID,STANDINGSDATE,CONFERENCE,TEAM,G,W,L,W_PCT,HOME_RECORD,ROAD_RECORD,RETURNTOPLAY
19302,16106127442020-12-28,1610612744,0,22020,2020-12-28,West,Golden State,3,1,2,0.333,0-0,1-2,NaN


In [ ]:
# Drop duplicates
ranking = ranking.drop_duplicates(subset=['ranking_PK'])

In [ ]:
# Make sure duplicates got dropped
ranking['ranking_PK'].value_counts()

ranking_PK
16106127432022-12-22    1
16106127652011-04-16    1
16106127412011-04-16    1
16106127482011-04-16    1
16106127382011-04-16    1
                       ..
16106127632007-12-25    1
16106127602007-12-25    1
16106127502007-12-25    1
16106127592007-12-24    1
16106127492014-09-01    1
Name: count, Length: 210313, dtype: int64

Limiting data to past five years (games_details)

In [ ]:
games_details

,games_details_PK,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,222004771629641,22200477,1610612759,SAS,San Antonio,1629641,Romeo Langford,Romeo,F,NaN,...,1.0,1.0,2.0,0.0,1.0,0.0,2.0,5.0,2.0,-2.0
1,222004771631110,22200477,1610612759,SAS,San Antonio,1631110,Jeremy Sochan,Jeremy,F,NaN,...,6.0,3.0,9.0,6.0,1.0,0.0,2.0,1.0,23.0,-14.0
2,222004771627751,22200477,1610612759,SAS,San Antonio,1627751,Jakob Poeltl,Jakob,C,NaN,...,1.0,3.0,4.0,1.0,1.0,0.0,2.0,4.0,13.0,-4.0
3,222004771630170,22200477,1610612759,SAS,San Antonio,1630170,Devin Vassell,Devin,G,NaN,...,0.0,9.0,9.0,5.0,3.0,0.0,2.0,1.0,10.0,-18.0
4,222004771630200,22200477,1610612759,SAS,San Antonio,1630200,Tre Jones,Tre,G,NaN,...,0.0,2.0,2.0,3.0,0.0,0.0,2.0,2.0,19.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668334,11200005202706,11200005,1610612743,DEN,Denver,202706,Jordan Hamilton,NaN,NaN,NaN,...,0.0,2.0,2.0,0.0,2.0,0.0,1.0,3.0,17.0,NaN
668335,11200005202702,11200005,1610612743,DEN,Denver,202702,Kenneth Faried,NaN,NaN,NaN,...,1.0,0.0,1.0,1.0,1.0,0.0,3.0,3.0,18.0,NaN
668336,11200005201585,11200005,1610612743,DEN,Denver,201585,Kosta Koufos,NaN,NaN,NaN,...,3.0,5.0,8.0,0.0,1.0,0.0,0.0,3.0,6.0,NaN
668337,11200005202389,11200005,1610612743,DEN,Denver,202389,Timofey Mozgov,NaN,NaN,NaN,...,1.0,2.0,3.0,1.0,0.0,0.0,4.0,2.0,2.0,NaN


Changing csvs containing columns with dates into datetime format

games.csv - 'GAME_DATE_EST'
ranking.csv - 'STANDINGSDATE'

In [ ]:
# Changing games.csv
games['GAME_DATE_EST'] = pd.to_datetime(games['GAME_DATE_EST'])

# Changing ranking.csv
ranking['STANDINGSDATE'] = pd.to_datetime(ranking['STANDINGSDATE'])

In [ ]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26622 entries, 0 to 26621
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   GAME_ID_pk        26622 non-null  int64         
 1   GAME_DATE_EST     26622 non-null  datetime64[ns]
 2   GAME_STATUS_TEXT  26622 non-null  object        
 3   HOME_TEAM_ID      26622 non-null  int64         
 4   VISITOR_TEAM_ID   26622 non-null  int64         
 5   SEASON            26622 non-null  int64         
 6   TEAM_ID_home      26622 non-null  int64         
 7   PTS_home          26523 non-null  float64       
 8   FG_PCT_home       26523 non-null  float64       
 9   FT_PCT_home       26523 non-null  float64       
 10  FG3_PCT_home      26523 non-null  float64       
 11  AST_home          26523 non-null  float64       
 12  REB_home          26523 non-null  float64       
 13  TEAM_ID_away      26622 non-null  int64         
 14  PTS_away          2652

In [ ]:
ranking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210313 entries, 0 to 210312
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   ranking_PK     210313 non-null  object        
 1   TEAM_ID        210313 non-null  int64         
 2   LEAGUE_ID      210313 non-null  int64         
 3   SEASON_ID      210313 non-null  int64         
 4   STANDINGSDATE  210313 non-null  datetime64[ns]
 5   CONFERENCE     210313 non-null  object        
 6   TEAM           210313 non-null  object        
 7   G              210313 non-null  int64         
 8   W              210313 non-null  int64         
 9   L              210313 non-null  int64         
 10  W_PCT          210313 non-null  float64       
 11  HOME_RECORD    210313 non-null  object        
 12  ROAD_RECORD    210313 non-null  object        
 13  RETURNTOPLAY   3990 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int64(6), object(5

In [ ]:
# Exporting cleaned data
ranking.to_csv('../data/cleaned_data/ranking_cleaned.csv',index=False,header=True)

Changing 'MINS' column into double, instead of str

In [ ]:
# Dropping all positions that aren't starters, to limit number of nulls
games_details = games_details.dropna(subset=['START_POSITION'])
games_details

,games_details_PK,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,222004771629641,22200477,1610612759,SAS,San Antonio,1629641,Romeo Langford,Romeo,F,NaN,...,1.0,1.0,2.0,0.0,1.0,0.0,2.0,5.0,2.0,-2.0
1,222004771631110,22200477,1610612759,SAS,San Antonio,1631110,Jeremy Sochan,Jeremy,F,NaN,...,6.0,3.0,9.0,6.0,1.0,0.0,2.0,1.0,23.0,-14.0
2,222004771627751,22200477,1610612759,SAS,San Antonio,1627751,Jakob Poeltl,Jakob,C,NaN,...,1.0,3.0,4.0,1.0,1.0,0.0,2.0,4.0,13.0,-4.0
3,222004771630170,22200477,1610612759,SAS,San Antonio,1630170,Devin Vassell,Devin,G,NaN,...,0.0,9.0,9.0,5.0,3.0,0.0,2.0,1.0,10.0,-18.0
4,222004771630200,22200477,1610612759,SAS,San Antonio,1630200,Tre Jones,Tre,G,NaN,...,0.0,2.0,2.0,3.0,0.0,0.0,2.0,2.0,19.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665670,21200001202087,21200001,1610612739,CLE,Cleveland,202087,Alonzo Gee,NaN,F,NaN,...,2.0,1.0,3.0,2.0,2.0,0.0,3.0,5.0,4.0,14.0
665671,21200001202684,21200001,1610612739,CLE,Cleveland,202684,Tristan Thompson,NaN,F,NaN,...,3.0,7.0,10.0,5.0,1.0,0.0,2.0,2.0,12.0,20.0
665672,212000012760,21200001,1610612739,CLE,Cleveland,2760,Anderson Varejao,NaN,C,NaN,...,12.0,11.0,23.0,9.0,0.0,2.0,1.0,4.0,9.0,7.0
665673,21200001203079,21200001,1610612739,CLE,Cleveland,203079,Dion Waiters,NaN,G,NaN,...,0.0,2.0,2.0,0.0,3.0,0.0,3.0,0.0,17.0,13.0


In [ ]:
# Dropping unnecessary columns that are making things more complicated by having nulls
games_details = games_details.drop(['COMMENT'], axis=1)
games_details = games_details.drop(['NICKNAME'],axis=1)

In [ ]:
# Checking for nulls
games_details.info()

<class 'pandas.core.frame.DataFrame'>
Index: 255644 entries, 0 to 665674
Data columns (total 28 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   games_details_PK   255644 non-null  object 
 1   GAME_ID            255644 non-null  int64  
 2   TEAM_ID            255644 non-null  int64  
 3   TEAM_ABBREVIATION  255644 non-null  object 
 4   TEAM_CITY          255644 non-null  object 
 5   PLAYER_ID          255644 non-null  int64  
 6   PLAYER_NAME        255644 non-null  object 
 7   START_POSITION     255644 non-null  object 
 8   MIN                255644 non-null  object 
 9   FGM                255644 non-null  float64
 10  FGA                255644 non-null  float64
 11  FG_PCT             255644 non-null  float64
 12  FG3M               255644 non-null  float64
 13  FG3A               255644 non-null  float64
 14  FG3_PCT            255644 non-null  float64
 15  FTM                255644 non-null  float64
 16  FTA    

In [ ]:
# Checking
games_details

,games_details_PK,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,MIN,FGM,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,222004771629641,22200477,1610612759,SAS,San Antonio,1629641,Romeo Langford,F,18:06,1.0,...,1.0,1.0,2.0,0.0,1.0,0.0,2.0,5.0,2.0,-2.0
1,222004771631110,22200477,1610612759,SAS,San Antonio,1631110,Jeremy Sochan,F,31:01,7.0,...,6.0,3.0,9.0,6.0,1.0,0.0,2.0,1.0,23.0,-14.0
2,222004771627751,22200477,1610612759,SAS,San Antonio,1627751,Jakob Poeltl,C,21:42,6.0,...,1.0,3.0,4.0,1.0,1.0,0.0,2.0,4.0,13.0,-4.0
3,222004771630170,22200477,1610612759,SAS,San Antonio,1630170,Devin Vassell,G,30:20,4.0,...,0.0,9.0,9.0,5.0,3.0,0.0,2.0,1.0,10.0,-18.0
4,222004771630200,22200477,1610612759,SAS,San Antonio,1630200,Tre Jones,G,27:44,7.0,...,0.0,2.0,2.0,3.0,0.0,0.0,2.0,2.0,19.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665670,21200001202087,21200001,1610612739,CLE,Cleveland,202087,Alonzo Gee,F,33:53,2.0,...,2.0,1.0,3.0,2.0,2.0,0.0,3.0,5.0,4.0,14.0
665671,21200001202684,21200001,1610612739,CLE,Cleveland,202684,Tristan Thompson,F,31:31,5.0,...,3.0,7.0,10.0,5.0,1.0,0.0,2.0,2.0,12.0,20.0
665672,212000012760,21200001,1610612739,CLE,Cleveland,2760,Anderson Varejao,C,37:22,3.0,...,12.0,11.0,23.0,9.0,0.0,2.0,1.0,4.0,9.0,7.0
665673,21200001203079,21200001,1610612739,CLE,Cleveland,203079,Dion Waiters,G,28:14,6.0,...,0.0,2.0,2.0,0.0,3.0,0.0,3.0,0.0,17.0,13.0


Chaning 'MINS' column to a float, but first need to change ; into . and fix all the mins that are the wrong format

In [ ]:
# Checking
games_details

,games_details_PK,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,MIN,FGM,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,222004771629641,22200477,1610612759,SAS,San Antonio,1629641,Romeo Langford,F,18:06,1.0,...,1.0,1.0,2.0,0.0,1.0,0.0,2.0,5.0,2.0,-2.0
1,222004771631110,22200477,1610612759,SAS,San Antonio,1631110,Jeremy Sochan,F,31:01,7.0,...,6.0,3.0,9.0,6.0,1.0,0.0,2.0,1.0,23.0,-14.0
2,222004771627751,22200477,1610612759,SAS,San Antonio,1627751,Jakob Poeltl,C,21:42,6.0,...,1.0,3.0,4.0,1.0,1.0,0.0,2.0,4.0,13.0,-4.0
3,222004771630170,22200477,1610612759,SAS,San Antonio,1630170,Devin Vassell,G,30:20,4.0,...,0.0,9.0,9.0,5.0,3.0,0.0,2.0,1.0,10.0,-18.0
4,222004771630200,22200477,1610612759,SAS,San Antonio,1630200,Tre Jones,G,27:44,7.0,...,0.0,2.0,2.0,3.0,0.0,0.0,2.0,2.0,19.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665670,21200001202087,21200001,1610612739,CLE,Cleveland,202087,Alonzo Gee,F,33:53,2.0,...,2.0,1.0,3.0,2.0,2.0,0.0,3.0,5.0,4.0,14.0
665671,21200001202684,21200001,1610612739,CLE,Cleveland,202684,Tristan Thompson,F,31:31,5.0,...,3.0,7.0,10.0,5.0,1.0,0.0,2.0,2.0,12.0,20.0
665672,212000012760,21200001,1610612739,CLE,Cleveland,2760,Anderson Varejao,C,37:22,3.0,...,12.0,11.0,23.0,9.0,0.0,2.0,1.0,4.0,9.0,7.0
665673,21200001203079,21200001,1610612739,CLE,Cleveland,203079,Dion Waiters,G,28:14,6.0,...,0.0,2.0,2.0,0.0,3.0,0.0,3.0,0.0,17.0,13.0


In [ ]:
games_details['MIN'] = games_details['MIN'].str.replace(":", ".")

In [ ]:
games_details['MIN'] = games_details['MIN'].str.replace(".000000.",".")

In [ ]:
games_details['MIN'] = games_details['MIN'].astype(float)

In [ ]:
games_details

,games_details_PK,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,MIN,FGM,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,222004771629641,22200477,1610612759,SAS,San Antonio,1629641,Romeo Langford,F,18.06,1.0,...,1.0,1.0,2.0,0.0,1.0,0.0,2.0,5.0,2.0,-2.0
1,222004771631110,22200477,1610612759,SAS,San Antonio,1631110,Jeremy Sochan,F,31.01,7.0,...,6.0,3.0,9.0,6.0,1.0,0.0,2.0,1.0,23.0,-14.0
2,222004771627751,22200477,1610612759,SAS,San Antonio,1627751,Jakob Poeltl,C,21.42,6.0,...,1.0,3.0,4.0,1.0,1.0,0.0,2.0,4.0,13.0,-4.0
3,222004771630170,22200477,1610612759,SAS,San Antonio,1630170,Devin Vassell,G,30.20,4.0,...,0.0,9.0,9.0,5.0,3.0,0.0,2.0,1.0,10.0,-18.0
4,222004771630200,22200477,1610612759,SAS,San Antonio,1630200,Tre Jones,G,27.44,7.0,...,0.0,2.0,2.0,3.0,0.0,0.0,2.0,2.0,19.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665670,21200001202087,21200001,1610612739,CLE,Cleveland,202087,Alonzo Gee,F,33.53,2.0,...,2.0,1.0,3.0,2.0,2.0,0.0,3.0,5.0,4.0,14.0
665671,21200001202684,21200001,1610612739,CLE,Cleveland,202684,Tristan Thompson,F,31.31,5.0,...,3.0,7.0,10.0,5.0,1.0,0.0,2.0,2.0,12.0,20.0
665672,212000012760,21200001,1610612739,CLE,Cleveland,2760,Anderson Varejao,C,37.22,3.0,...,12.0,11.0,23.0,9.0,0.0,2.0,1.0,4.0,9.0,7.0
665673,21200001203079,21200001,1610612739,CLE,Cleveland,203079,Dion Waiters,G,28.14,6.0,...,0.0,2.0,2.0,0.0,3.0,0.0,3.0,0.0,17.0,13.0


In [ ]:
# Checking that mins datatype changed
games_details.info()

<class 'pandas.core.frame.DataFrame'>
Index: 255644 entries, 0 to 665674
Data columns (total 28 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   games_details_PK   255644 non-null  object 
 1   GAME_ID            255644 non-null  int64  
 2   TEAM_ID            255644 non-null  int64  
 3   TEAM_ABBREVIATION  255644 non-null  object 
 4   TEAM_CITY          255644 non-null  object 
 5   PLAYER_ID          255644 non-null  int64  
 6   PLAYER_NAME        255644 non-null  object 
 7   START_POSITION     255644 non-null  object 
 8   MIN                255644 non-null  float64
 9   FGM                255644 non-null  float64
 10  FGA                255644 non-null  float64
 11  FG_PCT             255644 non-null  float64
 12  FG3M               255644 non-null  float64
 13  FG3A               255644 non-null  float64
 14  FG3_PCT            255644 non-null  float64
 15  FTM                255644 non-null  float64
 16  FTA    

Adding seasons to games_details.csv

In [ ]:
games[['SEASON','GAME_ID_pk']].rename(columns={'GAME_ID_pk':'GAME_ID'})

,SEASON,GAME_ID
0,2022,22200477
1,2022,22200478
2,2022,22200466
3,2022,22200467
4,2022,22200468
...,...,...
26617,2014,11400007
26618,2014,11400004
26619,2014,11400005
26620,2014,11400002


In [ ]:
# Get season and game_id columns from games.csv
games_seasons = games[['SEASON','GAME_ID_pk']].rename(columns={'GAME_ID_pk':'GAME_ID'})

# Merge games_seasons with games_details so that each game_id gets a season
games_details_szns = pd.merge(games_details,games_seasons, on='GAME_ID',how='left')
games_details_szns

,games_details_PK,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,MIN,FGM,...,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,SEASON
0,222004771629641,22200477,1610612759,SAS,San Antonio,1629641,Romeo Langford,F,18.06,1.0,...,1.0,2.0,0.0,1.0,0.0,2.0,5.0,2.0,-2.0,2022
1,222004771631110,22200477,1610612759,SAS,San Antonio,1631110,Jeremy Sochan,F,31.01,7.0,...,3.0,9.0,6.0,1.0,0.0,2.0,1.0,23.0,-14.0,2022
2,222004771627751,22200477,1610612759,SAS,San Antonio,1627751,Jakob Poeltl,C,21.42,6.0,...,3.0,4.0,1.0,1.0,0.0,2.0,4.0,13.0,-4.0,2022
3,222004771630170,22200477,1610612759,SAS,San Antonio,1630170,Devin Vassell,G,30.20,4.0,...,9.0,9.0,5.0,3.0,0.0,2.0,1.0,10.0,-18.0,2022
4,222004771630200,22200477,1610612759,SAS,San Antonio,1630200,Tre Jones,G,27.44,7.0,...,2.0,2.0,3.0,0.0,0.0,2.0,2.0,19.0,0.0,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255639,21200001202087,21200001,1610612739,CLE,Cleveland,202087,Alonzo Gee,F,33.53,2.0,...,1.0,3.0,2.0,2.0,0.0,3.0,5.0,4.0,14.0,2012
255640,21200001202684,21200001,1610612739,CLE,Cleveland,202684,Tristan Thompson,F,31.31,5.0,...,7.0,10.0,5.0,1.0,0.0,2.0,2.0,12.0,20.0,2012
255641,212000012760,21200001,1610612739,CLE,Cleveland,2760,Anderson Varejao,C,37.22,3.0,...,11.0,23.0,9.0,0.0,2.0,1.0,4.0,9.0,7.0,2012
255642,21200001203079,21200001,1610612739,CLE,Cleveland,203079,Dion Waiters,G,28.14,6.0,...,2.0,2.0,0.0,3.0,0.0,3.0,0.0,17.0,13.0,2012


Adding 2-pointer info for further analysis (games_details)

In [527]:
# Adding new columns for FG2
games_details_szns['FG2M'] = games_details_szns['FGM'] - games_details_szns['FG3M']
games_details_szns['FG2A'] = games_details_szns['FGA'] - games_details_szns['FG3A']
games_details_szns['FG2%'] = games_details_szns['FG2M'] / games_details_szns['FG2A']

In [528]:
# Double checking it worked...
games_details_szns

,games_details_PK,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,MIN,FGM,...,STL,BLK,TO,PF,PTS,PLUS_MINUS,SEASON,FG2M,FG2A,FG2%
0,222004771629641,22200477,1610612759,SAS,San Antonio,1629641,Romeo Langford,F,18.06,1.0,...,1.0,0.0,2.0,5.0,2.0,-2.0,2022,1.0,1.0,1.000000
1,222004771631110,22200477,1610612759,SAS,San Antonio,1631110,Jeremy Sochan,F,31.01,7.0,...,1.0,0.0,2.0,1.0,23.0,-14.0,2022,5.0,10.0,0.500000
2,222004771627751,22200477,1610612759,SAS,San Antonio,1627751,Jakob Poeltl,C,21.42,6.0,...,1.0,0.0,2.0,4.0,13.0,-4.0,2022,6.0,9.0,0.666667
3,222004771630170,22200477,1610612759,SAS,San Antonio,1630170,Devin Vassell,G,30.20,4.0,...,3.0,0.0,2.0,1.0,10.0,-18.0,2022,3.0,7.0,0.428571
4,222004771630200,22200477,1610612759,SAS,San Antonio,1630200,Tre Jones,G,27.44,7.0,...,0.0,0.0,2.0,2.0,19.0,0.0,2022,6.0,9.0,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255639,21200001202087,21200001,1610612739,CLE,Cleveland,202087,Alonzo Gee,F,33.53,2.0,...,2.0,0.0,3.0,5.0,4.0,14.0,2012,2.0,7.0,0.285714
255640,21200001202684,21200001,1610612739,CLE,Cleveland,202684,Tristan Thompson,F,31.31,5.0,...,1.0,0.0,2.0,2.0,12.0,20.0,2012,5.0,8.0,0.625000
255641,212000012760,21200001,1610612739,CLE,Cleveland,2760,Anderson Varejao,C,37.22,3.0,...,0.0,2.0,1.0,4.0,9.0,7.0,2012,3.0,7.0,0.428571
255642,21200001203079,21200001,1610612739,CLE,Cleveland,203079,Dion Waiters,G,28.14,6.0,...,3.0,0.0,3.0,0.0,17.0,13.0,2012,4.0,9.0,0.444444


In [529]:
# Exporting cleaned data
games_details_szns.to_csv('../data/cleaned_data/games_details_cleaned.csv',index=False,header=True)